<h1> Initialisation Cell</h1>

In [76]:
import numpy as np
import pandas as pd
import re

<h1> Importing the Data</h1>

In [77]:
dataset = pd.read_csv("logbook_assignment1.csv")
dataset.info()
dataset = np.array_split(dataset, 20)[0]


C:\Users\Mpendulo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (3,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174870 entries, 0 to 1174869
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   date_fueled      1125700 non-null  object
 1   date_captured    1174870 non-null  object
 2   odometer         1025716 non-null  object
 3   gallons          1100660 non-null  object
 4   cost_per_gallon  1094110 non-null  object
 5   total_spent      1100660 non-null  object
 6   mpg              1100660 non-null  object
 7   miles            146274 non-null   object
 8   user_url         1174870 non-null  object
dtypes: object(9)
memory usage: 80.7+ MB


<h1>Helper Functions</h1>

<h1>Data Cleaning</h1>

<h2>1.date_fueled</h2>

In [78]:
# Convert all invalid dates to NaT

regex = re.compile(r'[A-Z]{3} [0-9]{1,2} [0-9]{4}',re.I)
months = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07',
          'Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}

for i in range(dataset.shape[0]):
    test_string = str(dataset.loc[i,'date_fueled'])
    month = test_string[:3]
    if len(test_string) != 11 and len(test_string) != 10:
        dataset["date_fueled"].iloc[i] = 'NaT'
        continue
    if month in months.keys():
        update_string = test_string.replace(month,months[month])
    if bool(re.match(regex, test_string)) == False:
        dataset["date_fueled"].iloc[i] = 'NaT'
    if month in months.keys():   
        dataset["date_fueled"].iloc[i] = update_string
display(dataset["date_fueled"])



    





0               NaT
1         11 7 2012
2        09 22 2012
3         05 4 2019
4        02 15 2022
            ...    
58739           NaT
58740           NaT
58741     11 6 2021
58742     12 6 2018
58743     04 6 2022
Name: date_fueled, Length: 58744, dtype: object

In [79]:
t = '03fa'

tv = bool(re.match(regex, test_string))
print(tv)

True


In [80]:
# Set the dtype of column to datetime

dataset["date_fueled"] = pd.to_datetime(dataset["date_fueled"],format='%m %d %Y')
dataset["date_fueled"]

0              NaT
1       2012-11-07
2       2012-09-22
3       2019-05-04
4       2022-02-15
           ...    
58739          NaT
58740          NaT
58741   2021-11-06
58742   2018-12-06
58743   2022-04-06
Name: date_fueled, Length: 58744, dtype: datetime64[ns]

In [81]:
Sum = 0
for i in count.keys():
    Sum = Sum + int(count[i])

if Sum == dataset.shape[0]:
    print("All have been captured")

NameError: name 'count' is not defined

In [ ]:
for i in range(dataset["date_fueled"].shape[0]):
    if type(dataset["date_fueled"].iloc[i]) == type(float("nan")):
        dataset["date_fueled"].iloc[i] = "NaT"
        missing_values += 1

In [ ]:
missing_percentage = (missing_values/dataset["date_fueled"].shape[0])*100

print("The percentage of missing date fueled values = ",missing_percentage,"%")

<h2>2.Date_captured</h2>

<h2>3.Odometer</h2>

In [ ]:

length = dataset["odometer"].shape[0]
for index in range(length):
    entry = dataset["odometer"].iloc[index]
    if isinstance(entry,str):
        dataset["odometer"].iloc[index] = entry.replace(",","")
        print(dataset["odometer"].iloc[index])
        


<h2>4.Gallons</h2>

<h2>5.Cost per gallon</h2>

<h2>6.Total spent</h2>

<h2>7.Mile per gallon</h2>

<h2>8.Miles</h2>

<h1>Feature Engineering</h1>

<h1>Feature Selection</h1>